<a href="https://colab.research.google.com/github/zzickso/smart_factory/blob/main/2_%EC%8B%A4%ED%96%89_%EA%B0%80%EB%8A%A5%EC%84%B1_%ED%99%95%EC%9D%B8%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실행 가능성 확인하기

In [ ]:
pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2MB 29kB/s 
     |████████████████████████████████| 3.0MB 47.0MB/s 
     |████████████████████████████████| 460kB 51.3MB/s 
     |████████████████████████████████| 2.9MB 37.9MB/s 
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0


In [ ]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/busan_smart')
!ls

## 하이퍼파라미터

In [ ]:
EPOCHS = 10

DATASET_PATH = 'dataset/2/'
DATASET_OK_PATTERN = DATASET_PATH + 'OK/*.png'
DATASET_FAIL_PATTERN = DATASET_PATH + 'FAIL/*.png'

RESULT_SAVE_PATH = 'results/'

## 단순한 모델 설정

In [ ]:
def Model():
    return Sequential([Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)), #tensorflow 버전업으로 코드 변경
                       MaxPool2D(),
                       Conv2D(64, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(128, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(256, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Flatten(),
                       Dense(1, activation='sigmoid')])

## 데이터셋 불러오기

In [ ]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img, channels=1) #tensorflow 버전업으로 코드 변경
    return tf.image.convert_image_dtype(img, tf.float32)

In [ ]:
ok_list = glob.glob(DATASET_OK_PATTERN)
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

fail_list = glob.glob(DATASET_FAIL_PATTERN)
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Train, Valid 데이터셋 나누기

In [ ]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

## 모델 생성 및 학습

In [ ]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Epoch 1/10
31/31 [==============================] - 6s 180ms/step - loss: 0.3433 - accuracy: 0.9123 - val_loss: 0.3009 - val_accuracy: 0.9145
Epoch 2/10
31/31 [==============================] - 5s 167ms/step - loss: 0.3163 - accuracy: 0.9072 - val_loss: 0.3221 - val_accuracy: 0.9002
Epoch 3/10
31/31 [==============================] - 5s 165ms/step - loss: 0.2997 - accuracy: 0.9113 - val_loss: 0.2741 - val_accuracy: 0.9240
Epoch 4/10
31/31 [==============================] - 5s 169ms/step - loss: 0.2830 - accuracy: 0.9123 - val_loss: 0.2666 - val_accuracy: 0.9264
Epoch 5/10
31/31 [==============================] - 5s 164ms/step - loss: 0.3099 - accuracy: 0.9083 - val_loss: 0.2777 - val_accuracy: 0.9169
Epoch 6/10
31/31 [==============================] - 5s 164ms/step - loss: 0.2922 - accuracy: 0.9062 - val_loss: 0.2731 - val_accuracy: 0.9192
Epoch 7/10
31/31 [==============================] - 5s 166ms/step - loss: 0.2900 - accuracy: 0.9134 - val_loss: 0.2395 - val_accuracy: 0.9169
Epoch 

## 결과를 이미지로 저장

In [ ]:
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

index = 0
for imgs, labels in ds_valid:
    preds = model.predict(imgs)
    for idx in range(imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]
        
        if gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TN'
            
        cv2.imwrite(path + '/%.4f_%04d.png' % (y, index), imgs[idx].numpy() * 255)
        index +=1